# Social Graph Analysis Group Project

### Grandma’s Cookbook, Lookbook: An Analysis of Ingredients Association With Ethnic Cuisines

#### Group Members:
* Nick Potts
* Kristi Conry
* Jacob Lebowitz
* Anthony Bryson

### Possible Goals :)

1) Predict popular cuisine's based on what ingediants you have <br>
2) Essential ingrediants if you like a favorite cuisine <br>
3) Ingrediant recommendation system ( if you lke 3 types of cuisine find top similar ingrediants ) 

### Just some questions for our data :)

1) Which cuisine's are most similar based on ingediants? and why?

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import json
import collections
import re
import pickle

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [31]:
df = pd.read_csv('data/cleaned_data.csv', converters={'ingredient_list_cleaned': lambda x: x[1:-1].split(',')})

In [32]:
df.head()

,id,cuisine,ingredients,ingredients_sentence,ingredient_cleaned,ingredient_list_cleaned
0,10259,greek,"['romaine lettuce', 'black olives', 'grape tom...",romaine lettuce black olives grape tomatoes ga...,romain lettuc black oliv grape tomato garlic p...,"['romain lettuc', 'black oliv', 'grape tomat..."
1,25693,southern_us,"['plain flour', 'ground pepper', 'salt', 'toma...",plain flour ground pepper salt tomatoes ground...,plain flour ground pepper salt tomato ground b...,"['plain flour', 'ground pepper', 'salt', 't..."
2,20130,filipino,"['eggs', 'pepper', 'salt', 'mayonaise', 'cooki...",eggs pepper salt mayonaise cooking oil green c...,egg pepper salt mayonais cook oil green chili ...,"['egg', 'pepper', 'salt', 'mayonais', 'coo..."
3,22213,indian,"['water', 'vegetable oil', 'wheat', 'salt']",water vegetable oil wheat salt,water veget oil wheat salt,"['water', 'veget oil', 'wheat', 'salt']"
4,13162,indian,"['black pepper', 'shallots', 'cornflour', 'cay...",black pepper shallots cornflour cayenne pepper...,black pepper shallot cornflour cayenn pepper o...,"['black pepper', 'shallot', 'cornflour', 'c..."


In [33]:
cleaned_df['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [34]:
real_ingredients = []

for ingredients in df['ingredient_list_cleaned']:
    for ingredient in ingredients:
        real_ingredients.append(ingredient.strip())
        
count_of_ingredient = collections.Counter(real_ingredients)
count_of_ingredient.most_common(10)

[("'salt'", 18049),
 ("'onion'", 7972),
 ("'oliv oil'", 7972),
 ("'water'", 7457),
 ("'garlic'", 7380),
 ("'sugar'", 6434),
 ("'garlic clove'", 6237),
 ("'butter'", 4848),
 ("'ground black pepper'", 4785),
 ("'allpurpos flour'", 4632)]

In [35]:
for ingredients in df['ingredient_list_cleaned'][:5]:
    print(type(ingredients))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
